In [2]:
import sys
sys.path.append('..')
from src.kitti import *
import numpy as np
import cv2
import json
import glob
from src.boundingbox import format_yolo_output
from src.trajectory import Trajectory

%matplotlib inline

In [ ]:
LEFTDIR = '/data/urbinn/datasets/kitti/KITTI_Left_Images/training/image_2'
RIGHTDIR = '/data/urbinn/datasets/kitti/KITTI_Right_Images/training/image_3'

In [ ]:
yolo_data = format_yolo_output("data/yolo/25000.json")
image_path = "{}/007300.png".format(LEFTDIR)

image = read_image(image_path)
frame = Frame(image_path, RIGHTDIR, yolo_data[0])

In [ ]:
# Create frame from one image; in kitti sequence 02
# - Get observations
# - Filter observations by bounding box boundings
# - Plot to verify result
#

In [ ]:
s = Sequence()
s.add_frame(frame)

In [ ]:
# Sets classification for observation
frame.update_observations_per_classification()

# Filters observations without classifications
frame.filter_no_classification()

# We're only interested in mappoints
frame.filter_non_mappoint()

show(draw_observations_classification(frame.get_observations()))

### Analyzing KITTI sequence 00 image_2

In [3]:
LEFTDIR = '/data/urbinn/datasets/kitti/sequences/00/image_2'
RIGHTDIR = '/data/urbinn/datasets/kitti/sequences/00/image_3'

yolo_data = format_yolo_output("/data/urbinn/darknet/output/seq00_image02/objects.json")
                
frames = []
for filename in sorted(glob.glob(LEFTDIR + '/*')): 
    image_name = filename.split('/')[-1]
    frame = None
    
    if not image_name in yolo_data:
        print("frame {} has no yolo data available".format(image_name))
        frame = Frame(filename, RIGHTDIR)
    else:
        frame = Frame(filename, RIGHTDIR, yolo_data[image_name])
        
    frames.append(frame)

frame 004540.png has no yolo data available


Note: For some reason frame 004540 has no objects detected. This might be possible but requires further investigation.

#### Setting up a sequence

In [7]:
seq = Sequence()
for f in ProgressBar()(frames[:5]):
    seq.add_frame(f)

100% (5 of 5) |###########################| Elapsed Time: 0:00:06 Time: 0:00:06


In [6]:
DIMENSION = 400

poses = [kf.get_pose() for kf in seq.keyframes]
trajectory = Trajectory(poses, (DIMENSION, DIMENSION))

../src/trajectory.py:25: RuntimeWarning: invalid value encountered in double_scalars
  x = self.dimension[0] * (p[0] - minx) / (maxxy)
../src/trajectory.py:26: RuntimeWarning: invalid value encountered in double_scalars
  z = self.dimension[0] - self.dimension[1] * (p[2] - miny) / (maxxy)


In order to plot the observations it is necessary to convert them to real world coordinates.